In [1]:
import pickle
import os
import copy
import re
import time
import importlib

import numpy as np

In [2]:
import random

In [3]:
with open('AlCoCrCuFeMnNiV_5_10.pkl', 'rb') as f:
    structure_DB = pickle.load(f)

In [4]:
structure_DB

,init_structure,size
0,"(Atom('Al', [0.0, 0.0, 0.0], index=0))",1
1,"(Atom('Co', [0.0, 0.0, 0.0], index=0))",1
2,"(Atom('Cr', [0.0, 0.0, 0.0], index=0))",1
3,"(Atom('Cu', [0.0, 0.0, 0.0], index=0))",1
4,"(Atom('Fe', [0.0, 0.0, 0.0], index=0))",1
...,...,...
509933,"(Atom('Cu', [0.0, 0.0, 0.0], index=0), Atom('C...",10
509934,"(Atom('Cu', [0.0, 0.0, 0.0], index=0), Atom('C...",10
509935,"(Atom('Cu', [0.0, 0.0, 0.0], index=0), Atom('C...",10
509936,"(Atom('Cu', [0.0, 0.0, 0.0], index=0), Atom('C...",10


In [5]:
def delete_files(file_list):
    """Delete multiple files."""
    for file_path in file_list:
        try:
            os.remove(file_path)
            print(f"Deleted: {file_path}")
        except FileNotFoundError:
            print(f"File not found: {file_path}")
        except PermissionError:
            print(f"Permission denied: {file_path}")
        except Exception as e:
            print(f"Error deleting {file_path}: {e}")

In [6]:
import disall

In [7]:
%load_ext autoreload
%autoreload 2

In [8]:
importlib.reload(disall)

<module 'disall' (<_frozen_importlib_external._NamespaceLoader object at 0x151493b514b0>)>

In [9]:
from disall.grid_run import run_packet
from disall.pyiron_calculator import VASP_pyiron_calculator, read_unfinished_job, _zip_job_dir

2025-01-31 06:13:50,778 - pyiron_log - WARNING - pyiron found a 'templates' folder in the /scratch/group/arroyave_lab/guillermo.vazquez/pyiron_dirs/pyiron/resources resource directory. These are no longer supported in pyiron_base >=0.7.0. They are replaced by Project.create_job_class() and Project.wrap_python_function().


## how to calculate vasp relaxer
it needs to be an starter or checker.
if not started then it starts it, if running, checks. Maybe different functions

In [10]:
# df = structure_DB.copy(deep = True)

In [11]:
with open('first_reread.pkl', 'rb') as f:
    df = pickle.load(f)

In [12]:
# df [df['first_00'].isnull()]

In [13]:
# delete_files(['first_00.log','first_00.pkl'])
# !rm -r first_00

In [14]:
run_packet?

Signature:
run_packet(
    df_name,
    df_global,
    relaxer_dict,
    input_data=None,
    num_cores=10,
    max_tries=5,
    **kwargs,
)
Docstring:
why we shouldn't run two of these at the same time?
if the numbers are repeated they would be looping the same numbers flagging a job node as occupied when 
two run_packet shouldn't never be run at the same time so the database doesn't handle 
two different loops saving and loading! (only to o when a changed occured?)
File:      /scratch/user/guillermo.vazquez/SAVE/octonary/disall/disall/grid_run.py
Type:      function

In [15]:
input_data = {'-INCAR-NCORE': 16,
    '-INCAR-LORBIT': 1,
    '-INCAR-NSW': 200,
    '-INCAR-PREC': 'Accurate',
    '-INCAR-IBRION': 2,
    '-INCAR-ISMEAR': 0,
    '-INCAR-SIGMA': 0.02,
    '-INCAR-ENCUT': 400,
    '-INCAR-EDIFF': 1E-5,
    '-INCAR-ISIF': 3,
    '-INCAR-ISYM': 0,
    'KPPA': 4000,
    '-INCAR-ISPIN': 2,
    }

In [16]:
relaxer_dic_timeout = {'calculator_label': 'first_00',
                            'relaxer': VASP_pyiron_calculator}

In [17]:
ids = np.arange(980).tolist()

In [18]:
df.loc[ids]

,init_structure,size,first_00
0,"(Atom('Al', [0.0, 0.0, 0.0], index=0))",1,"{'master_status': 'FINISHED', 'error': '', 'st..."
1,"(Atom('Co', [0.0, 0.0, 0.0], index=0))",1,"{'master_status': 'FINISHED', 'error': '', 'st..."
2,"(Atom('Cr', [0.0, 0.0, 0.0], index=0))",1,"{'master_status': 'FINISHED', 'error': '', 'st..."
3,"(Atom('Cu', [0.0, 0.0, 0.0], index=0))",1,"{'master_status': 'FINISHED', 'error': '', 'st..."
4,"(Atom('Fe', [0.0, 0.0, 0.0], index=0))",1,"{'master_status': 'FINISHED', 'error': '', 'st..."
...,...,...,...
975,"(Atom('Cu', [0.0, 0.0, 0.0], index=0), Atom('N...",4,"{'master_status': 'FINISHED', 'job_name': 'vas..."
976,"(Atom('Cu', [0.0, 0.0, 0.0], index=0), Atom('N...",4,"{'master_status': 'FINISHED', 'job_name': 'vas..."
977,"(Atom('Cu', [0.0, 0.0, 0.0], index=0), Atom('N...",4,"{'master_status': 'FINISHED', 'job_name': 'vas..."
978,"(Atom('Cu', [0.0, 0.0, 0.0], index=0), Atom('N...",4,"{'master_status': 'FINISHED', 'job_name': 'vas..."


In [19]:
# df = run_packet('first_reread',
#         df,
#         relaxer_dic_timeout,
#         input_data = copy.deepcopy(input_data),
#         id_list = ids,
#         num_cores = 980,
#         max_tries = 5)

In [20]:
# df = run_packet('first_reread',
#         df,
#         relaxer_dic_timeout,
#         input_data = copy.deepcopy(input_data),
#         id_list = ids,
#         num_cores = 20,
#         max_tries = 5)

In [74]:
ids = np.arange(1000,1100).tolist()
for i in range(20):
    try:
        df = run_packet('first_reread',
                df,
                relaxer_dic_timeout,
                input_data = copy.deepcopy(input_data),
                id_list = ids,
                num_cores = 20,
                max_tries = 5)
        time.sleep(30)
    except Exception as e:
        print('EXCEPTION : ', e)
        time.sleep(30)

in 20 number of cores
##########< i: 0, to_calculate[i]: 1000  >##########
this calculation has finished succesfully, already parsed
##########< i: 1, to_calculate[i]: 1001  >##########
this calculation has finished succesfully, already parsed
##########< i: 2, to_calculate[i]: 1002  >##########
this calculation has finished succesfully, already parsed
##########< i: 3, to_calculate[i]: 1003  >##########
this calculation has finished succesfully, already parsed
##########< i: 4, to_calculate[i]: 1004  >##########


2025-01-30 09:36:45,002 - pyiron_log - WARNING - Could not access indices, returning None!


pyiron_status start of loop: finished
pyiron QID 1976301
status_pyiron: finished, status_slurm: COMPLETED
parsing pyiron job...
master_status FINISHED
this calculation has finished succesfully, parsing master dictionary...
##########< i: 5, to_calculate[i]: 1005  >##########
this calculation has finished succesfully, already parsed
##########< i: 6, to_calculate[i]: 1006  >##########
this calculation has finished succesfully, already parsed
##########< i: 7, to_calculate[i]: 1007  >##########
this calculation has finished succesfully, already parsed
##########< i: 8, to_calculate[i]: 1008  >##########
this calculation has finished succesfully, already parsed
##########< i: 9, to_calculate[i]: 1009  >##########
this calculation has finished succesfully, already parsed
##########< i: 10, to_calculate[i]: 1010  >##########
this calculation has finished succesfully, already parsed
##########< i: 11, to_calculate[i]: 1011  >##########
this calculation has finished succesfully, already parse

2025-01-30 09:36:46,053 - pyiron_log - WARNING - Could not access indices, returning None!


pyiron_status start of loop: finished
pyiron QID 1976509
status_pyiron: finished, status_slurm: COMPLETED
parsing pyiron job...
master_status FINISHED
this calculation has finished succesfully, parsing master dictionary...
##########< i: 45, to_calculate[i]: 1045  >##########
pyiron_status start of loop: running
pyiron QID 1976639
status_pyiron: running, status_slurm: RUNNING
master_status RUNNING
##########< i: 46, to_calculate[i]: 1046  >##########
pyiron_status start of loop: running
pyiron QID 1976640
status_pyiron: running, status_slurm: RUNNING
master_status RUNNING
##########< i: 47, to_calculate[i]: 1047  >##########
pyiron_status start of loop: running
pyiron QID 1976641
status_pyiron: running, status_slurm: RUNNING
master_status RUNNING
##########< i: 48, to_calculate[i]: 1048  >##########
pyiron_status start of loop: running
pyiron QID 1976642
status_pyiron: running, status_slurm: RUNNING
master_status RUNNING
##########< i: 49, to_calculate[i]: 1049  >##########
pyiron_stat

...

pyiron_status start of loop: finished
pyiron QID 1976665
status_pyiron: finished, status_slurm: COMPLETED
parsing pyiron job...
master_status FINISHED
this calculation has finished succesfully, parsing master dictionary...
##########< i: 70, to_calculate[i]: 1070  >##########
pyiron_status start of loop: running
pyiron QID 1976666
status_pyiron: running, status_slurm: RUNNING
master_status RUNNING
##########< i: 71, to_calculate[i]: 1071  >##########
pyiron_status start of loop: running
pyiron QID 1976667
status_pyiron: running, status_slurm: RUNNING
master_status RUNNING
##########< i: 72, to_calculate[i]: 1072  >##########
this calculation has finished succesfully, already parsed
##########< i: 73, to_calculate[i]: 1073  >##########
pyiron_status start of loop: running
pyiron QID 1976670
status_pyiron: running, status_slurm: RUNNING
master_status RUNNING
##########< i: 74, to_calculate[i]: 1074  >##########


2025-01-30 09:54:27,342 - pyiron_log - WARNING - Could not access indices, returning None!


pyiron_status start of loop: finished
pyiron QID 1976672
status_pyiron: finished, status_slurm: COMPLETED
parsing pyiron job...
master_status FINISHED
this calculation has finished succesfully, parsing master dictionary...
##########< i: 75, to_calculate[i]: 1075  >##########
pyiron_status start of loop: running
pyiron QID 1976673
status_pyiron: running, status_slurm: RUNNING
master_status RUNNING
##########< i: 76, to_calculate[i]: 1076  >##########
pyiron_status start of loop: running
pyiron QID 1976674
status_pyiron: running, status_slurm: RUNNING
master_status RUNNING
##########< i: 77, to_calculate[i]: 1077  >##########
pyiron_status start of loop: running
pyiron QID 1976675
status_pyiron: running, status_slurm: RUNNING
master_status RUNNING
##########< i: 78, to_calculate[i]: 1078  >##########
pyiron_status start of loop: running
pyiron QID 1976676
status_pyiron: running, status_slurm: RUNNING
master_status RUNNING
##########< i: 79, to_calculate[i]: 1079  >##########
pyiron_stat

2025-01-30 09:54:35,451 - pyiron_log - WARNING - Job aborted


EXCEPTION :  run_queue.sh crashed


In [25]:
ids = np.arange(1000,1100).tolist()
for i in range(1):
    try:
        df = run_packet('first_reread',
                df,
                relaxer_dic_timeout,
                input_data = copy.deepcopy(input_data),
                id_list = ids,
                num_cores = 20,
                max_tries = 5)
        time.sleep(30)
    except Exception as e:
        print('EXCEPTION : ', e)
        time.sleep(30)

in 20 number of cores
##########< i: 0, to_calculate[i]: 1000  >##########
this calculation has finished succesfully, already parsed
##########< i: 1, to_calculate[i]: 1001  >##########
this calculation has finished succesfully, already parsed
##########< i: 2, to_calculate[i]: 1002  >##########
this calculation has finished succesfully, already parsed
##########< i: 3, to_calculate[i]: 1003  >##########
this calculation has finished succesfully, already parsed
##########< i: 4, to_calculate[i]: 1004  >##########
this calculation has finished succesfully, already parsed
##########< i: 5, to_calculate[i]: 1005  >##########
this calculation has finished succesfully, already parsed
##########< i: 6, to_calculate[i]: 1006  >##########
this calculation has finished succesfully, already parsed
##########< i: 7, to_calculate[i]: 1007  >##########
this calculation has finished succesfully, already parsed
##########< i: 8, to_calculate[i]: 1008  >##########
this calculation has finished succesf

2025-01-30 10:36:20,475 - pyiron_log - WARNING - Could not access indices, returning None!


status_pyiron: finished, status_slurm: COMPLETED
parsing pyiron job...
master_status FINISHED
this calculation has finished succesfully, parsing master dictionary...
##########< i: 85, to_calculate[i]: 1085  >##########
this calculation has finished succesfully, already parsed
##########< i: 86, to_calculate[i]: 1086  >##########
this calculation has finished succesfully, already parsed
##########< i: 87, to_calculate[i]: 1087  >##########
this calculation has finished succesfully, already parsed
##########< i: 88, to_calculate[i]: 1088  >##########
this calculation has finished succesfully, already parsed
##########< i: 89, to_calculate[i]: 1089  >##########
this calculation has finished succesfully, already parsed
##########< i: 90, to_calculate[i]: 1090  >##########
this calculation has finished succesfully, already parsed
##########< i: 91, to_calculate[i]: 1091  >##########
this calculation has finished succesfully, already parsed
##########< i: 92, to_calculate[i]: 1092  >#######

In [22]:
# unmarked_at_4 = np.arange(df.attrs['first_00']['marked'][-1] + 1, df[df['size'] == 4].index.tolist()[-1] + 1).tolist()

In [23]:
# np.arange(1100, df[df['size'] == 4].index.tolist()[-1] + 1)

In [29]:
# df.to_pickle('df_1100'+'.pkl')

In [18]:
unmarked_at_4 = np.arange(1100, df[df['size'] == 4].index.tolist()[-1] + 1).tolist()

In [19]:
random.seed(42)
test_ids = random.sample(unmarked_at_4, 500)

In [20]:
test_ids.sort()

In [21]:

for i in range(1):
    try:
        df = run_packet('first_reread',
                df,
                relaxer_dic_timeout,
                input_data = copy.deepcopy(input_data),
                id_list = test_ids,
                num_cores = 20,
                max_tries = 5)
        time.sleep(30)
    except Exception as e:
        print('EXCEPTION : ', e)
        time.sleep(30)

in 20 number of cores
##########< i: 0, to_calculate[i]: 1102  >##########
this calculation has finished succesfully, already parsed
##########< i: 1, to_calculate[i]: 1108  >##########
this calculation has finished succesfully, already parsed
##########< i: 2, to_calculate[i]: 1113  >##########
this calculation has finished succesfully, already parsed
##########< i: 3, to_calculate[i]: 1114  >##########
this calculation has finished succesfully, already parsed
##########< i: 4, to_calculate[i]: 1126  >##########
this calculation has finished succesfully, already parsed
##########< i: 5, to_calculate[i]: 1129  >##########
this calculation has finished succesfully, already parsed
##########< i: 6, to_calculate[i]: 1132  >##########
this calculation has finished succesfully, already parsed
##########< i: 7, to_calculate[i]: 1138  >##########
this calculation has finished succesfully, already parsed
##########< i: 8, to_calculate[i]: 1139  >##########
this calculation has finished succesf

In [29]:

for i in range(1):
    try:
        df = run_packet('first_reread',
                df,
                relaxer_dic_timeout,
                input_data = copy.deepcopy(input_data),
                id_list = test_ids,
                num_cores = 20,
                max_tries = 5)
        time.sleep(30)
    except Exception as e:
        print('EXCEPTION : ', e)
        time.sleep(30)

in 20 number of cores
##########< i: 0, to_calculate[i]: 1102  >##########
this calculation has finished succesfully, already parsed
##########< i: 1, to_calculate[i]: 1108  >##########
this calculation has finished succesfully, already parsed
##########< i: 2, to_calculate[i]: 1113  >##########
this calculation has finished succesfully, already parsed
##########< i: 3, to_calculate[i]: 1114  >##########
this calculation has finished succesfully, already parsed
##########< i: 4, to_calculate[i]: 1126  >##########
this calculation has finished succesfully, already parsed
##########< i: 5, to_calculate[i]: 1129  >##########
this calculation has finished succesfully, already parsed
##########< i: 6, to_calculate[i]: 1132  >##########
this calculation has finished succesfully, already parsed
##########< i: 7, to_calculate[i]: 1138  >##########
this calculation has finished succesfully, already parsed
##########< i: 8, to_calculate[i]: 1139  >##########
this calculation has finished succesf

2025-01-31 15:53:08,853 - pyiron_log - WARNING - Could not access indices, returning None!


pyiron_status start of loop: finished
pyiron QID 1984586
status_pyiron: finished, status_slurm: COMPLETED
parsing pyiron job...
master_status FINISHED
this calculation has finished succesfully, parsing master dictionary...
##########< i: 313, to_calculate[i]: 3261  >##########
this calculation has finished succesfully, already parsed
##########< i: 314, to_calculate[i]: 3266  >##########
this calculation has finished succesfully, already parsed
##########< i: 315, to_calculate[i]: 3267  >##########
this calculation has finished succesfully, already parsed
##########< i: 316, to_calculate[i]: 3272  >##########
this calculation has finished succesfully, already parsed
##########< i: 317, to_calculate[i]: 3275  >##########
this calculation has finished succesfully, already parsed
##########< i: 318, to_calculate[i]: 3281  >##########
this calculation has finished succesfully, already parsed
##########< i: 319, to_calculate[i]: 3287  >##########
this calculation has finished succesfully, a

2025-01-31 15:53:09,426 - pyiron_log - WARNING - Could not access indices, returning None!


pyiron_status start of loop: finished
pyiron QID 1984618
status_pyiron: finished, status_slurm: COMPLETED
parsing pyiron job...
master_status FINISHED
this calculation has finished succesfully, parsing master dictionary...
##########< i: 332, to_calculate[i]: 3351  >##########
this calculation has finished succesfully, already parsed
##########< i: 333, to_calculate[i]: 3354  >##########


...

pyiron_status start of loop: finished
pyiron QID 1984677
status_pyiron: finished, status_slurm: COMPLETED
parsing pyiron job...
master_status FINISHED
this calculation has finished succesfully, parsing master dictionary...
##########< i: 349, to_calculate[i]: 3433  >##########


2025-01-31 15:53:19,197 - pyiron_log - WARNING - Could not access indices, returning None!


pyiron_status start of loop: finished
pyiron QID 1984678
status_pyiron: finished, status_slurm: COMPLETED
parsing pyiron job...
master_status FINISHED
this calculation has finished succesfully, parsing master dictionary...
##########< i: 350, to_calculate[i]: 3438  >##########


2025-01-31 15:53:19,750 - pyiron_log - WARNING - Could not access indices, returning None!


pyiron_status start of loop: finished
pyiron QID 1984679
status_pyiron: finished, status_slurm: COMPLETED
parsing pyiron job...
master_status FINISHED
this calculation has finished succesfully, parsing master dictionary...
##########< i: 351, to_calculate[i]: 3460  >##########
pyiron_status start of loop: initialized
PASS to RUN --> id000000000003460
SENDING...
The job vasp_000000000003460 was saved and received the ID: 29793
Queue system id:  1984761
##########< i: 352, to_calculate[i]: 3464  >##########
pyiron_status start of loop: initialized
PASS to RUN --> id000000000003464
SENDING...
The job vasp_000000000003464 was saved and received the ID: 29794
Queue system id:  1984762
##########< i: 353, to_calculate[i]: 3471  >##########
pyiron_status start of loop: initialized
PASS to RUN --> id000000000003471
SENDING...
The job vasp_000000000003471 was saved and received the ID: 29795
Queue system id:  1984763
##########< i: 354, to_calculate[i]: 3472  >##########
pyiron_status start of 

In [30]:

for i in range(20):
    try:
        df = run_packet('first_reread',
                df,
                relaxer_dic_timeout,
                input_data = copy.deepcopy(input_data),
                id_list = test_ids,
                num_cores = 20,
                max_tries = 5)
        time.sleep(30)
    except Exception as e:
        print('EXCEPTION : ', e)
        time.sleep(30)

in 20 number of cores
##########< i: 0, to_calculate[i]: 1102  >##########
this calculation has finished succesfully, already parsed
##########< i: 1, to_calculate[i]: 1108  >##########
this calculation has finished succesfully, already parsed
##########< i: 2, to_calculate[i]: 1113  >##########
this calculation has finished succesfully, already parsed
##########< i: 3, to_calculate[i]: 1114  >##########
this calculation has finished succesfully, already parsed
##########< i: 4, to_calculate[i]: 1126  >##########
this calculation has finished succesfully, already parsed
##########< i: 5, to_calculate[i]: 1129  >##########
this calculation has finished succesfully, already parsed
##########< i: 6, to_calculate[i]: 1132  >##########
this calculation has finished succesfully, already parsed
##########< i: 7, to_calculate[i]: 1138  >##########
this calculation has finished succesfully, already parsed
##########< i: 8, to_calculate[i]: 1139  >##########
this calculation has finished succesf

2025-01-31 15:59:57,866 - pyiron_log - WARNING - Could not access indices, returning None!


pyiron_status start of loop: finished
pyiron QID 1984762
status_pyiron: finished, status_slurm: COMPLETED
parsing pyiron job...
master_status FINISHED
this calculation has finished succesfully, parsing master dictionary...
##########< i: 353, to_calculate[i]: 3471  >##########


2025-01-31 15:59:58,326 - pyiron_log - WARNING - Could not access indices, returning None!


pyiron_status start of loop: finished
pyiron QID 1984763
status_pyiron: finished, status_slurm: COMPLETED
parsing pyiron job...
master_status FINISHED
this calculation has finished succesfully, parsing master dictionary...
##########< i: 354, to_calculate[i]: 3472  >##########


...

pyiron_status start of loop: finished
pyiron QID 1984777
status_pyiron: finished, status_slurm: COMPLETED
parsing pyiron job...
master_status FINISHED
this calculation has finished succesfully, parsing master dictionary...
##########< i: 368, to_calculate[i]: 3565  >##########
pyiron_status start of loop: running
pyiron QID 1984778
status_pyiron: running, status_slurm: RUNNING
master_status RUNNING
##########< i: 369, to_calculate[i]: 3572  >##########
pyiron_status start of loop: running
pyiron QID 1984779
status_pyiron: running, status_slurm: RUNNING
master_status RUNNING
##########< i: 370, to_calculate[i]: 3577  >##########
pyiron_status start of loop: running
pyiron QID 1984780
status_pyiron: running, status_slurm: RUNNING
master_status RUNNING
##########< i: 371, to_calculate[i]: 3584  >##########
pyiron_status start of loop: running
pyiron QID 1984781
status_pyiron: running, status_slurm: RUNNING
master_status RUNNING
##########< i: 372, to_calculate[i]: 3594  >##########
pyiron

2025-01-31 16:11:01,159 - pyiron_log - WARNING - Could not access indices, returning None!


pyiron_status start of loop: finished
pyiron QID 1984769
status_pyiron: finished, status_slurm: COMPLETED
parsing pyiron job...
master_status FINISHED
this calculation has finished succesfully, parsing master dictionary...
##########< i: 360, to_calculate[i]: 3510  >##########
this calculation has finished succesfully, already parsed
##########< i: 361, to_calculate[i]: 3513  >##########
this calculation has finished succesfully, already parsed
##########< i: 362, to_calculate[i]: 3518  >##########
pyiron_status start of loop: running
pyiron QID 1984772
status_pyiron: running, status_slurm: RUNNING
master_status RUNNING
##########< i: 363, to_calculate[i]: 3535  >##########
this calculation has finished succesfully, already parsed
##########< i: 364, to_calculate[i]: 3537  >##########
this calculation has finished succesfully, already parsed
##########< i: 365, to_calculate[i]: 3540  >##########
this calculation has finished succesfully, already parsed
##########< i: 366, to_calculate[

2025-01-31 16:13:33,102 - pyiron_log - WARNING - Could not access indices, returning None!


pyiron_status start of loop: finished
pyiron QID 1984776
status_pyiron: finished, status_slurm: COMPLETED
parsing pyiron job...
master_status FINISHED
this calculation has finished succesfully, parsing master dictionary...
##########< i: 367, to_calculate[i]: 3553  >##########
this calculation has finished succesfully, already parsed
##########< i: 368, to_calculate[i]: 3565  >##########
pyiron_status start of loop: running
pyiron QID 1984778
status_pyiron: running, status_slurm: RUNNING
master_status RUNNING
##########< i: 369, to_calculate[i]: 3572  >##########
pyiron_status start of loop: running
pyiron QID 1984779
status_pyiron: running, status_slurm: RUNNING
master_status RUNNING
##########< i: 370, to_calculate[i]: 3577  >##########
pyiron_status start of loop: running
pyiron QID 1984780
status_pyiron: running, status_slurm: RUNNING
master_status RUNNING
##########< i: 371, to_calculate[i]: 3584  >##########
pyiron_status start of loop: running
pyiron QID 1984781
status_pyiron: r